In [26]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
import numpy as np
import matplotlib.pyplot as plt

In [27]:
#Datos artificiales

import numpy as np
import pandas as pd

np.random.seed(42)
n = 25  # 24 clientes + 1 depósito

# Crear matriz de distancias aleatorias simétrica
dist_matrix_np = np.random.uniform(2, 25, size=(n, n))
dist_matrix_np = (dist_matrix_np + dist_matrix_np.T) / 2

# Convertir a diccionario {(i,j): distancia}
distancia = {(i, j): dist_matrix_np[i][j] for i in range(n) for j in range(n)}

vehiculos = {
  1: (130, 170),
  2: (140, 200),
  3: (120, 180),
  4: (100, 90),
  5: (70, 100),
  6: (55, 170),
  7: (110, 150),
  8: (114, 140)
}

demanda = {
  0:0, 1: 13, 2: 15, 3: 12, 4: 15, 5: 20, 6: 17, 7: 17, 8: 20, 9: 20, 10: 15,
  11: 17, 12: 12, 13: 21, 14: 15, 15: 17, 16: 10, 17: 25, 18: 12,
  19: 11, 20: 15, 21: 14, 22: 18, 23: 15, 24: 11
}




In [28]:
model = ConcreteModel()

# Conjuntos e índices

model.N = Set(initialize=range(0, n))  # Todos los nodos (0 = depósito)
model.C = Set(initialize=demanda.keys())  # Clientes
model.V = Set(initialize=vehiculos.keys())  # Vehículos
model.D = Set(initialize=[0])  # Nodo depósito


# Variables
model.x = Var(model.N, model.N, model.V, domain=Binary)  # Ruta (i->j) por vehículo v
model.u = Var(model.N, model.V, domain=NonNegativeReals)  # Carga acumulada en nodo i por vehículo v

# Parámetros

model.dist = Param(model.N, model.N, initialize=distancia, default=0)
model.demand = Param(model.C, initialize=demanda)
model.cap = Param(model.V, initialize={v: vehiculos[v][0] for v in vehiculos})
model.range = Param(model.V, initialize={v: vehiculos[v][1] for v in vehiculos})

# Función Objetivo

def obj_rule(m):
    return sum(m.dist[i, j] * m.x[i, j, v] for i in m.N for j in m.N if i != j for v in m.V)
model.obj = Objective(rule=obj_rule, sense=minimize)


# Restricciones

# Cada cliente es visitado exactamente una vez
model.visit_once = ConstraintList()
for c in model.C:
    model.visit_once.add(
        sum(model.x[i, c, v] for i in model.N if i != c for v in model.V) == 1
    )

# Flujo conservado por vehículo en cada cliente
model.flow_conservation = ConstraintList()
for v in model.V:
    for c in model.C:
        model.flow_conservation.add(
            sum(model.x[i, c, v] for i in model.N if i != c) ==
            sum(model.x[c, j, v] for j in model.N if j != c)
        )

# Salida del depósito por vehículo es máximo 1
model.depot_departure = ConstraintList()
for v in model.V:
    model.depot_departure.add(
        sum(model.x[0, j, v] for j in model.N if j != 0) <= 1
    )

# Capacidad del vehículo no debe excederse
model.capacity_constraint = ConstraintList()
for v in model.V:
    model.capacity_constraint.add(
        sum(model.demand[i] * model.x[i, j, v] for i in model.C for j in model.N if i != j) <= model.cap[v]
    )
# Valor inicial de carga en el depósito es 0
model.initial_load = ConstraintList()
for v in model.V:
    model.initial_load.add(model.u[0, v] == 0)

# La carga nunca puede superar la capacidad del vehículo
model.max_capacity = ConstraintList()
for v in model.V:
    for i in model.N:
        model.max_capacity.add(model.u[i, v] <= model.cap[v])

# Rango del vehículo no debe excederse
model.range_constraint = ConstraintList()
for v in model.V:
    model.range_constraint.add(
        sum(model.dist[i, j] * model.x[i, j, v] for i in model.N for j in model.N if i != j) <= model.range[v]
    )

# No permitir ciclos triviales (i -> i)
model.no_loops = ConstraintList()
for v in model.V:
    for i in model.N:
        model.no_loops.add(model.x[i, i, v] == 0)


opt = SolverFactory('glpk')
results = opt.solve(model, tee=False)

print("Valor de la función objetivo:", model.obj())




Valor de la función objetivo: 127.36132834715629
